<a href="https://colab.research.google.com/github/rithikafjohnson/Autonomous-Farming-System-Crop-Analysis/blob/main/fcm_%2B_som.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-fuzzy

     |████████████████████████████████| 1.0MB 27.6MB/s 
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-cp37-none-any.whl size=894069 sha256=accfce65e37891dfa78e8e4dd0418f5e49ee636279e2bfe7407a011e61fee620
  Stored in directory: /root/.cache/pip/wheels/b9/4e/77/da79b16f64ef1738d95486e2731eea09d73e90a72465096600
Successfully built scikit-fuzzy


In [ ]:
!pip install minisom

  Created wheel for minisom: filename=MiniSom-2.2.9-cp37-none-any.whl size=8603 sha256=8d1354b473a4573f3671db0244b79aa2cd357a766d0a82bc88a463a07e830f3e
  Stored in directory: /root/.cache/pip/wheels/de/a0/08/5234d6b02b29c561f62b6c985e2eb7d480fb0b92359a8c74e4
Successfully built minisom


In [ ]:
!unzip RiceDiseases-DataSet-master.zip

Archive:  RiceDiseases-DataSet-master.zip
   creating: RiceDiseases-DataSet-master/
   creating: RiceDiseases-DataSet-master/Bacterial leaf blight/
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_001.jpg  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_002.png  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_003.jpg  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_004.PNG  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_005.jpg  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_006.PNG  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_007.JPG  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_008.PNG  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_009.JPG  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_010.PNG  
  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skfuzzy as fuzz
import os
import cv2
import numpy as np
from time import time
from PIL import Image
from numpy import asarray
%matplotlib inline

In [ ]:
def change_color_fuzzycmeans(cluster_membership, clusters):
    img = []
    for pix in cluster_membership.T:
        img.append(clusters[np.argmax(pix)])
    return img
clabs=[]
fdict = {new_list: [] for new_list in range(3)}
def readimage():
    folder ='RiceDiseases-DataSet-master/'
    list_images = os.listdir(folder)
    list_img = []
    for i in list_images:
        
        list2=os.listdir(folder+i)
        for image in list2:
          im=(folder+i+'/'+image)
          clabs.append(i)
          img = cv2.imread(im)
          rgb_img = img.reshape((img.shape[0] * img.shape[1], 3))
          list_img.append(rgb_img)
        
        
    return list_img 

    

        
        
    
    
      
def bwarea(img):
    row = img.shape[0]
    col = img.shape[1]
    total = 0.0
    for r in range(row-1):
        for c in range(col-1):
            sub_total = img[r:r+2, c:c+2].mean()
            if sub_total == 255:
                total += 1
            elif sub_total == (255.0/3.0):
                total += (7.0/8.0)
            elif sub_total == (255.0/4.0):
                total += 0.25
            elif sub_total == 0:
                total += 0
            else:
                r1c1 = img[r,c]
                r1c2 = img[r,c+1]
                r2c1 = img[r+1,c]
                r2c2 = img[r+1,c+1]
                
                if (((r1c1 == r2c2) & (r1c2 == r2c1)) & (r1c1 != r2c1)):
                    total += 0.75
                else:
                    total += 0.5
    return total
            
def imclearborder(imgBW):

    # Given a black and white image, first find all of its contours
    radius = 2
    imgBWcopy = imgBW.copy()
    contours,hierarchy = cv2.findContours(imgBWcopy.copy(), cv2.RETR_LIST, 
        cv2.CHAIN_APPROX_SIMPLE)

    # Get dimensions of image
    imgRows = imgBW.shape[0]
    imgCols = imgBW.shape[1]    

    contourList = [] # ID list of contours that touch the border

    # For each contour...
    for idx in np.arange(len(contours)):
        # Get the i'th contour
        cnt = contours[idx]

        # Look at each point in the contour
        for pt in cnt:
            rowCnt = pt[0][1]
            colCnt = pt[0][0]

            # If this is within the radius of the border
            # this contour goes bye bye!
            check1 = (rowCnt >= 0 and rowCnt < radius) or (rowCnt >= imgRows-1-radius and rowCnt < imgRows)
            check2 = (colCnt >= 0 and colCnt < radius) or (colCnt >= imgCols-1-radius and colCnt < imgCols)

            if check1 or check2:
                contourList.append(idx)
                break

    for idx in contourList:
        cv2.drawContours(imgBWcopy, contours, idx, (0,0,0), -1)

    return imgBWcopy

#### bwareaopen definition
def bwareaopen(imgBW, areaPixels):
    # Given a black and white image, first find all of its contours
    imgBWcopy = imgBW.copy()
    contours, hierarchy = cv2.findContours(imgBWcopy.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # For each contour, determine its total occupying area
    for idx in np.arange(len(contours)):
        area = cv2.contourArea(contours[idx])
        if (area >= 0 and area <= areaPixels):
            cv2.drawContours(imgBWcopy, contours, idx, (0,0,0), -1)

    return imgBWcopy      

def imfill(im_th):
    
    im_floodfill = im_th.copy()
    # Mask used to flood filling.
    
    # Notice the size needs to be 2 pixels than the image.
    h, w = im_th.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    # Floodfill from point (0, 0
    cv2.floodFill(im_floodfill, mask, (0,0), 255);

    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)

    # Combine the two images to get the foreground.
    im_out = im_th | im_floodfill_inv
    
    return im_out

In [ ]:
list_img = readimage()


n_data = len(list_img)
imglst=[]

In [ ]:
clusters = []

# looping every images
for index,rgb_img in enumerate(list_img):
    img = np.reshape(rgb_img, (300,300,3)).astype(np.uint8)
    shape = np.shape(img)
    
    
    # initialize graph
    plt.figure(figsize=(20,20))
    plt.subplot(1,4,1)
    plt.imshow(img)
    # looping every cluster     
    print('Image '+str(index+1))
    for i,cluster in enumerate(clusters):
            
        # Fuzzy C Means
        new_time = time()
        
        # error = 0.005
        # maximum iteration = 1000
        # cluster = 2,3,6,8
        
        cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
        rgb_img.T, cluster, 2, error=0.005, maxiter=1000, init=None,seed=42)

        new_img = change_color_fuzzycmeans(u,cntr)
        
        fuzzy_img = np.reshape(new_img,shape).astype(np.uint8)
        
        ret, seg_img = cv2.threshold(fuzzy_img,np.max(fuzzy_img),255,cv2.THRESH_BINARY)
        
        print('Fuzzy time for cluster',cluster)
        print(time() - new_time,'seconds')
        seg_img_1d = seg_img[:,:,1]
        
        
        bwfim1 = bwareaopen(seg_img_1d, 100)
        bwfim2 = imclearborder(bwfim1)
        bwfim3 = imfill(bwfim2)
        
        
        

  
       
        print('Bwarea : '+str(bwarea(bwfim3)))
        print()

        plt.subplot(1,4,i+2)
        np_im = np.array(bwfim3)
        name = 'Cluster'+str(cluster)
        plt.title(name)

    name = 'segmented'+str(index)+'.png'
   

    np_im = np.array(img)
    b, g, r = cv2.split(img)
    pixels = g/1.
    np.save(('nparr{}').format(index), pixels)
    fdict[0].append(name)
    fdict[2].append(clabs[index-1])
    fdict[1].append(('nparr{}.npy').format(index))
    plt.savefig(name)
    
 
    

Image 1
Image 2
Image 3
Image 4
Image 5
Image 6
Image 7
Image 8
Image 9
Image 10
Image 11
Image 12
Image 13
Image 14
Image 15
Image 16
Image 17
Image 18
Image 19
Image 20


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # Remove the CWD from sys.path while we load stuff.


Image 21
Image 22
Image 23
Image 24
Image 25
Image 26
Image 27
Image 28
Image 29
Image 30
Image 31
Image 32
Image 33
Image 34
Image 35
Image 36
Image 37
Image 38
Image 39
Image 40
Image 41
Image 42
Image 43
Image 44
Image 45
Image 46
Image 47
Image 48
Image 49
Image 50
Image 51
Image 52
Image 53
Image 54
Image 55
Image 56
Image 57
Image 58
Image 59
Image 60
Image 61
Image 62
Image 63
Image 64


SystemError: ignored

Error in callback <function flush_figures at 0x7f0ef916f440> (for post_execute):


KeyboardInterrupt: ignored

In [ ]:
import csv
keys = sorted(fdict.keys())
with open("test.csv", "w") as outfile:
   writer = csv.writer(outfile, delimiter = ",")
   writer.writerow(keys)
   writer.writerows(zip(*[fdict[key] for key in keys]))

In [ ]:
dataset=pd.read_csv('test.csv')
dataset.head()

In [ ]:
y=dataset.iloc[:,2].values

In [ ]:
x=dataset.iloc[:,1].values

In [ ]:
from minisom import MiniSom


In [ ]:
va=0
while va<len(x):
  with open(x[va], 'rb') as f:
    x[va] = np.load(f)
  va=va+1
  

In [ ]:
imglst

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report





X_train, X_test, y_train, y_test = train_test_split(imglst, y, test_size=0.25)


som = MiniSom(300, 300, 300 , sigma=3, learning_rate=0.5, random_seed=10 , neighborhood_function='triangle')



 

#som.random_weights_init(imglst)
som.train_random(X_train, 25)


print(classification_report(y_test, classify(som, X_test)))

In [ ]:
from hyperopt import fmin, tpe, hp
xx=np.vstack(x)

In [ ]:
best=fmin(fn= lambda sig :  MiniSom(300, 300, 300 , sigma=3, learning_rate=0.5, random_seed=10 , neighborhood_function='triangle').quantization_error(xx),space=hp.uniform('sig', 0.001, 300/2.01),algo=tpe.suggest, max_evals=200 )

In [ ]:
som.train_random(X_train, 25)

In [ ]:
from hyperopt import Trials, STATUS_OK

In [ ]:
space={'sig': hp.uniform('sig', 0.001, 5 ), 'learning_rate': hp.uniform('learning_rate', 0.001, 5)}

In [ ]:
def som_fn(space):
  sig=space['sig']
  learning_rate=space['learning_rate']
  val=MiniSom(300, 300, 300 , sigma=sig, learning_rate=learning_rate, random_seed=10 , neighborhood_function='triangle').quantization_error(x)
  return{'loss': val, 'status':STATUS_OK }

In [ ]:
trials=Trials()
best=fmin(fn=som_fn,space=space,algo=tpe.suggest,max_evals=100, trials=trials)

for i trial in enumerate(trials.trials[:2]):
  print(i,trial)

In [ ]:
sigma=best['sig']
learning_rate=best['learning_rate']

In [ ]:
som = MiniSom(300, 300, 300 , sigma, learning_rate, random_seed=10 , neighborhood_function='triangle')

In [ ]:
som.train_random(X_train, 25)

In [ ]:
print(classification_report(y_test, classify(som, X_test)))